In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import matplotlib.pyplot as plt

from nma_class import NMA_project

%matplotlib inline

In [ ]:
# only if you are on google coalb not if you are on jupyter notebook
from google.colab import drive
drive.mount('/content/gdrive')

In [3]:
# only if you are on jupyter notebook not if on colab
import qgrid

In [4]:
# example google colab load session
#main_folder = "/content/gdrive/My Drive/3 Projekte/Neuromatch Summer School/Projekt/steinmetz_dataset"
#main_folder = "C:/Users/User/Google Drive/3 Projekte/Neuromatch Summer School/Projekt/steinmetz_dataset"
main_folder = "Z:/3 Projekte/Neuromatch Summer School/Projekt/steinmetz_dataset"

# create session object
nma_obj = NMA_project(main_folder) 

# default session is initialized, extract interesting dataframes in dedicated variables
session = nma_obj.std_session
channels_df = session['channels_df']
spikes_df = session['spikes_df']
clusters_df = session['clusters_df']


In [5]:
qgrid.show_grid(channels_df)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [7]:
qgrid.show_grid(clusters_df)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

## Load Data


### via oneibl by client

In [5]:

from oneibl.onelight import ONE
one = ONE()
one.set_figshare_url("https://figshare.com/articles/steinmetz/9974357")


ModuleNotFoundError: ignored

In [ ]:
sessions = one.search(['spikes'])


### via google drive directly

In [3]:
def load_session(folder):
    """
    load all files from given folder to dict
    return dict with key as filenames
    """
    # load session files ===========================================
    from os import listdir
    from os.path import isfile, join
    files = [f for f in listdir(folder) if isfile(join(folder, f))]
    names = ['_'.join(f.split('.')[:-1]) for f in files]
    session = dict()
    for (file_, name_) in zip(files, names):
        if file_.split('.')[-1] == 'npy':
        session[name_] = np.load(folder+'/'+file_)
        if file_.split('.')[-1] == 'tsv':
        session[name_] = pd.read_table(folder+'/'+file_)

    # pars session info two dataframe ===============================
    # create spike_df tataframe, with each spike time and the cluster it belongs to
    spikes_df = pd.DataFrame( { 'cluster':session['spikes_clusters'][:,0], 'spike_times': session['spikes_times'][:,0] }, ) 

    # create cluster dataframe information and spikes for each cluster
    clusters = np.unique(session['spikes_clusters'])
    # create number of spikes, phy2 manual cluster, 
    # parse phy annotation
    clusters_df = pd.DataFrame({'lable':session['clusters__phy_annotation'][:,0], 'probe':session['clusters_probes'][:,0], 'peak channel':session['clusters_peakChannel'][:,0]}, index=clusters )
    # pars labels in strings 1 = good 3 = mua
    #clusters_df['lable'] = clusters_df['lable'].apply( lambda lable: 'good' if lable==3 else ('mua' if lable==1 else 'bad') )

    # create spikes colum with spiketimes
    spk = pd.DataFrame( {'spikes':np.zeros(len(clusters), dtype=object)}, index=clusters )
    for group, frame in spikes_df.groupby('cluster'):
        spk['spikes'][group] = frame['spike_times'].values

    #merge spike column with clusters_df
    clusters_df = pd.merge(clusters_df, spk, how='right', left_index=True, right_index=True)
    # set index name
    clusters_df.index.name='cluster'
    # number of spikes per cluster
    clusters_df['total spikes'] = clusters_df['spikes'].apply(lambda row: len(row) )

# def load_session(folder, sessions):
#   """
#   load all files in each given session and add to session dict
#   returs a dict of sessions, with each session a dict of files
#   """
#   sessions_data = dict()
#   for session in sessions:
#     current_session = dict()
#     sessions[session] = 

In [7]:
# example google colab load session
#folder = "/content/gdrive/My Drive/3 Projekte/Neuromatch Summer School/Projekt/steinmetz_dataset"
#folder = "C:/Users/User/Google Drive/3 Projekte/Neuromatch Summer School/Projekt/steinmetz_dataset"
folder = "Z:/3 Projekte/Neuromatch Summer School/Projekt/steinmetz_dataset"

subfolders = [ f.path for f in os.scandir(folder) if f.is_dir() ]
sessions = [folder.split('/')[-1] for folder in subfolders]

# load first session
session = load_session(subfolders[0])

KeyboardInterrupt: 

In [10]:
folder = "Z:/3 Projekte/Neuromatch Summer School/Projekt/steinmetz_dataset/Cori_2016-12-14"
session = load_session(folder)

In [11]:
for key in session.keys():
  print(key)

Cori_2016-12-14_M2_g0_t0_imec_lf_timestamps
Cori_2016-12-14_V1_g0_t0_imec_lf_timestamps
channels_brainLocation
channels_probe
channels_rawRow
channels_site
channels_sitePositions
clusters__phy_annotation
clusters_depths
clusters_originalIDs
clusters_peakChannel
clusters_probes
clusters_templateWaveformChans
clusters_templateWaveforms
clusters_waveformDuration
eye_area
eye_timestamps
eye_xyPos
face_motionEnergy
face_timestamps
lickPiezo_raw
lickPiezo_timestamps
licks_times
passiveBeeps_times
passiveValveClick_times
passiveVisual_contrastLeft
passiveVisual_contrastRight
passiveVisual_times
passiveWhiteNoise_times
probes_description
probes_insertion
probes_rawFilename
probes_sitePositions
sparseNoise_positions
sparseNoise_times
spikes_amps
spikes_clusters
spikes_depths
spikes_times
spontaneous_intervals
trials_feedbackType
trials_feedback_times
trials_goCue_times
trials_included
trials_intervals
trials_repNum
trials_response_choice
trials_response_times
trials_visualStim_contrastLeft
tria

### cleanup channel infos

In [76]:
# load channel brain location infos
brain = pd.DataFrame(session['channels_brainLocation'])
# load channel probes
site = pd.DataFrame(session['channels_sitePositions'], columns=['channel 0', 'channel 1'])
# merge with channles_df
channels_df = pd.merge(brain, site, how='inner', left_index=True, right_index=True)
# load probe, row and site
meta = pd.DataFrame({'probe':session['channels_probe'][:,0],
                            'raw row':session['channels_rawRow'][:,0],
                            'channels_site':session['channels_site'][:,0] })
# merge metha with channel_df
channels_df = pd.merge(channels_df, meta, how='inner', left_index=True, right_index=True)

In [77]:
qgrid.show_grid(channels_df)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

### cleaning up spikes

In [12]:
# create spike_df tataframe, with each spike time and the cluster it belongs to
spikes_df = pd.DataFrame( { 'cluster':session['spikes_clusters'][:,0], 'spike_times': session['spikes_times'][:,0] }, ) 
qgrid.show_grid(spikes_df)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [55]:
# create cluster dataframe information and spikes for each cluster
clusters = np.unique(session['spikes_clusters'])
# create number of spikes, phy2 manual cluster, 
# parse phy annotation
clusters_df = pd.DataFrame({'lable':session['clusters__phy_annotation'][:,0], 'probe':session['clusters_probes'][:,0], 'peak channel':session['clusters_peakChannel'][:,0]}, index=clusters )
# pars labels in strings 1 = good 3 = mua
#clusters_df['lable'] = clusters_df['lable'].apply( lambda lable: 'good' if lable==3 else ('mua' if lable==1 else 'bad') )



# create spikes colum with spiketimes
spk = pd.DataFrame( {'spikes':np.zeros(len(clusters), dtype=object)}, index=clusters )
for group, frame in spikes_df.groupby('cluster'):
    spk['spikes'][group] = frame['spike_times'].values

#merge spike column with clusters_df
clusters_df = pd.merge(clusters_df, spk, how='right', left_index=True, right_index=True)
# set index name
clusters_df.index.name='cluster'
# number of spikes per cluster
clusters_df['total spikes'] = clusters_df['spikes'].apply(lambda row: len(row) )

In [56]:
qgrid.show_grid(clusters_df)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

### cleanup trial info